FFT
$$
X\omega (t) = \int_{-\infty}^{\infty} x(t) e^{-j\omega t}dt
$$

IFFT
$$
x(t) = \frac{a}{2\pi}\int_{-\infty}^{\infty} X(t) e^{-j\omega t} dt
$$

hanning
$$
x(t) = 0.5 - 0.5 \cos 2\pi t, \; ( 0 \leq t \leq 1 )
$$

hamming
$$
x(t) = 0.54 - 0.46 \cos 2 \pi t, \; (0 \leq t \leq 1)
$$

blackman
$$
x(t) = 0.42 - 0.5 \cos 2 \pi t + 0.08 \cos 4 \pi t, \; (0 \leq t \leq 1)
$$

rectangular
$$
x(t) = 1, \; (0 \leq t \leq 1)
$$

In [1]:
from FFT_Calculation2 import MyFFT
import pandas as pd
from ipywidgets import widgets, Layout
import plotly.graph_objects as go

In [3]:
# FFT Calculation
myfft = MyFFT()

data = pd.read_csv('sample_data_for_FFT.csv')
t = data.iloc[:, 0]
x = data.iloc[:, 1]

# dt = 0.01 # サンプリング周期
# split_t_r = 0.1 # 窓の範囲
# overlap = 0.5 # 窓関数のオーバーラップ
# window_F = 'hanning'
output_FN = 'test.png'
y_label = 'amplitude'
y_unit = 'V'

#######################################################################################################
# Create Widgets

# サンプリング周期
dt_btn = widgets.FloatLogSlider(value=0.01, min=-4, max=-1, step=1, 
                                description='dt:', 
                                layout=Layout(width='80%'))

# 窓関数の選択
window_F_btn = widgets.Dropdown(value='rectangular', description='window:', options=myfft.window_list)
# 窓の範囲
# 何分割するか．0分割=1. 10分割=0.1．
split_t_r_btn = widgets.FloatLogSlider(value=0.1, min=-2, max=0, step=1, description='窓の範囲:',
                                       layout=dt_btn.layout)
# 窓関数のオーバーラップ
overlap_btn = widgets.FloatSlider(value=0.5, min=0.1, max=1, step=0.1, description='Overlap:',
                                  layout=dt_btn.layout)
# t Range Slider
range_slider = widgets.FloatRangeSlider(value=[0, len(t)-1], min=0, max=len(t)-1, step=1,
                                        description='t Range Slider:', 
                                        readout_format='d', 
                                        style = {'description_width': 'initial'}, 
                                        layout=Layout(width='90%'))

#######################################################################################################
# Substitute returned values for variables
fq_avg, F_abs_amp_avg = myfft.FFT_main(t, x, dt_btn.value, split_t_r_btn.value, overlap_btn.value, 
                                       window_F_btn.value, output_FN, y_label, y_unit)

#######################################################################################################
# Assign an empty figure widget with a trace
trace1 = go.Scatter(x=t, y=x, name='raw data')

graph1 = go.FigureWidget(data=trace1, 
                         layout=go.Layout(title=dict(text='Sample data'),
                                          xaxis=dict(title='Time (s)'),
                                          yaxis=dict(title=y_label + '(' + y_unit + ')'),
                                          height=300, width=900))

trace2 = go.Scatter(x=fq_avg, y=F_abs_amp_avg, name=window_F_btn.value)

graph2 = go.FigureWidget(data=trace2, 
                         layout=go.Layout(title=dict(text='FFT'),
                                          xaxis=dict(title='Frequency (Hz)'),
                                          yaxis=dict(title=y_label + '( ' + y_unit + '/sqrt(Hz))'),
                                          height=400, width=900))

#######################################################################################################
# Write functions that will handle the input from the widgets
def response(change):
    X0 = int(range_slider.value[0])
    X1 = int(range_slider.value[1])
    fq_avg, F_abs_amp_avg = myfft.FFT_main(t[X0:X1], x[X0:X1], dt_btn.value, split_t_r_btn.value, overlap_btn.value, 
                                           window_F_btn.value, output_FN, y_label, y_unit)

    with graph2.batch_update():        
        graph1.data[0].x = t[X0:X1]
        graph1.data[0].y = x[X0:X1]
        graph2.data[0].x = fq_avg
        graph2.data[0].y = F_abs_amp_avg
        
dt_btn.observe(response, names="value")
window_F_btn.observe(response, names="value")
split_t_r_btn.observe(response, names="value")
overlap_btn.observe(response, names="value")
range_slider.observe(response, names='value')

# Widgets Box
btn_widgets = widgets.VBox([window_F_btn, dt_btn, split_t_r_btn, overlap_btn, range_slider])

app = widgets.VBox([btn_widgets, graph1, graph2])
app